In [1]:
from Scripts.utils import *  
import pandas as pd

columns = ['Date','Open','High','Low','Close','Volume','Yield','PercentageVolume',
           'SMA6','EMA6','WMA6','HMA6','SMA20','EMA20','WMA20','HMA20','SMA50','EMA50','WMA50','HMA50',
           'SMA100','EMA100','WMA100','HMA100','MACD','CCI','Stochastic Oscillator','RSI','ROC','PPO',
           'KST','BOLU','BOLD','BOLM']
data = pd.read_csv('DataReady/EURUSD/EURUSD_D1.csv', names = columns, header = 0)

In [2]:
toRemove = ['Volume', 'Date','High','Low','Open','Close']
df = selectData(data,toRemove)

In [3]:
normDf = normalizeData(df)
images = generateImages(normDf)
images = np.array(images)

In [4]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(images, data['Close'][28:], test_size = 0.2,shuffle = False)
train_X, valid_X, train_Y, valid_Y = train_test_split(train_X, train_Y, test_size = 0.1,shuffle = False)

In [5]:
train_X = train_X.reshape(train_X.shape[0],28,28).astype(np.float32)
valid_X = valid_X.reshape(valid_X.shape[0],28,28).astype(np.float32)
test_X = test_X.reshape(test_X.shape[0],28,28).astype(np.float32)

In [6]:
import torch 
train_X, valid_X, test_X = torch.from_numpy(train_X).type(torch.FloatTensor), torch.from_numpy(valid_X).type(torch.FloatTensor), torch.from_numpy(test_X).type(torch.FloatTensor)
train_Y, valid_Y, test_Y = torch.from_numpy(np.array(train_Y)).type(torch.FloatTensor), torch.from_numpy(np.array(valid_Y)).type(torch.FloatTensor), torch.from_numpy(np.array(test_Y)).type(torch.FloatTensor)

In [7]:
from Scripts.BayesianRNN import *

model = BayesianRNN()
loss_fn = torch.nn.MSELoss(reduction='sum')
loss_module = LossModule(model,(train_X, train_Y), loss_fn)

In [ ]:
SGHMC = SGHMCSampler(loss_module, num_burn_in_steps=5000, lr=0.003, keep_every = 100, mdecay=0.01)
SGHMC.sample(nsamples=1000)
sampled_weights = SGHMC.sampled_weights